In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
warnings.filterwarnings(action='ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'

import time
from tqdm.notebook import tqdm

import datetime
from datetime import timedelta

from datetime import datetime
import re

In [64]:
total = pd.read_excel('data.xlsx')
aa = pd.read_excel('aadata.xlsx')
bb = pd.read_excel('bbdata.xlsx')

In [65]:
del aa["Unnamed: 0"]
del bb["Unnamed: 0"]
total.drop(["Unnamed: 0"],axis=1,inplace=True)

In [66]:
total = pd.concat([total,aa])
total = pd.concat([total,bb])
total.drop(1100,inplace=True)
total = total.reset_index(drop=True)

In [67]:
total["날짜"].fillna(method='ffill',inplace=True)
total["부문"].fillna(method='ffill',inplace=True)
total["신문사"].fillna(method='ffill',inplace=True)
total["제목"].fillna(method='ffill',inplace=True)

### 데이터 병합

In [68]:
df1= pd.read_excel("NewsResult_20200420-20200601.xlsx")
df2= pd.read_excel("NewsResult_20200602-20200715.xlsx")
df3= pd.read_excel("NewsResult_20200715-20200826.xlsx")

In [69]:
total.columns

Index(['날짜', '본문', '부문', '신문사', '제목'], dtype='object')

In [70]:
df1 = df1[["일자","본문","통합 분류1","언론사","제목"]]
df2 = df2[["일자","본문","통합 분류1","언론사","제목"]]
df3 = df3[["일자","본문","통합 분류1","언론사","제목"]]

In [71]:
df1.rename(columns= {'일자':'날짜','통합 분류1':'부문','언론사':'신문사'},inplace=True)
df2.rename(columns= {'일자':'날짜','통합 분류1':'부문','언론사':'신문사'},inplace=True)
df3.rename(columns={'일자':'날짜','통합 분류1':'부문','언론사':'신문사'},inplace=True)

In [72]:
total.dropna(inplace=True)
total = total.reset_index(drop=True)

In [73]:
total = pd.concat([total,df1])
total = pd.concat([total,df2])
total = pd.concat([total,df3])

In [75]:
total.dropna(inplace=True)

### 날짜 전처리

- 중복수집된 데이터 제거 (제목,본문을 기준으로)
- 다른 컬럼은 중복될 수 밖에 없는 컬럼

In [ ]:
total.drop_duplicates('제목',inplace=True)
total.drop_duplicates('본문',inplace=True)

In [95]:
total = total.reset_index(drop=True)

- 정수형,문자열 날짜의 형식 표준화 '2020-02-01'

In [125]:
for i in tqdm(range(len(total))):
    if type(total["날짜"][i]) == str:
        total["날짜"][i] = total["날짜"][i].replace("/","-")
    elif type(total["날짜"][i]) == int:
        total["날짜"][i] = str(total["날짜"][i])[:4]+"-"+ str(total["날짜"][i])[4:6]+"-"+str(total["날짜"][i])[6:]
    elif type(total["날짜"][i]) == float:
        total["날짜"][i] = str(total["날짜"][i])[:4]+"-"+ str(total["날짜"][i])[4:6]+"-"+str(total["날짜"][i])[6:8]
    else:
        continue

- 문자열을 날짜형식으로 변환

In [126]:
for i in tqdm(range(len(total))):
    if type(total["날짜"][i]) == str:
        total["날짜"][i] = datetime.strptime(total["날짜"][i], "%Y-%m-%d")
    else:
        continue

In [136]:
for i in range(len(total)):
    total["날짜"][i] = str(total["날짜"][i])[:10]

In [138]:
total = total.sort_values(by='날짜').reset_index(drop=True) # 정렬

### 부문 전처리

In [140]:
q =[]
for i in range(len(total)):
    q.append(total["부문"][i][:2])

In [146]:
pre = pd.DataFrame(q, columns=['부문'])

In [148]:
for i in range(len(pre)):
    if pre["부문"][i] == '스포':
        pre["부문"][i] = '스포츠'
    elif pre["부문"][i] =='미분':
        pre["부문"][i] = '미분류'
    else:
        continue

In [ ]:
del total['부문']

In [154]:
total['부문'] = pre['부문']

In [156]:
total = total[['날짜','부문','신문사','제목','본문']]

In [157]:
total

,날짜,부문,신문사,제목,본문
0,2020-02-01,문화,조선일보,끔찍하면서도 훌륭한 타이밍…전염병과의 전쟁 외,"[아무튼, 주말- saturday's pick]\n넷플릭스 | 판데믹\n\n""이런 ..."
1,2020-02-01,IT,서울신문,"전문가 충격진단 ""신종코로나,사스 4배 이러다간…""",신종 코로나바이러스 감염증(신종코로나) 유행이 세계 경제에 미칠 타격이 2003년 ...
2,2020-02-01,지역,조선일보,'우한 폐렴' 격리·직무배제로 생계곤란 시 긴급지원·유급휴가 받는다,정부가 ‘우한 폐렴’(신종 코로나바이러스 감염증)환자와의 접촉에 따른 감염전파 위험...
3,2020-02-01,국제,국민일보,"신종 코로나 12번째 확진자, 수원 소재 친적집 방문… 수원시 긴급 방역나서",부천시에 거주 중인 신종 코로나바이러스 감염증(이하 신종 코로나) 12번째 확진자가...
4,2020-02-01,정치,중앙일보,"""신종코로나 후폭? 사스 4배 충격…세계경제에 191조원 타격""",[연합뉴스] [헤럴드경제] 신종 코로나바이러스 감염증(신종코로나) 폭증으로 세계 경...
...,...,...,...,...,...
50667,2020-08-26,정치,내일신문,내년도 ‘슈퍼예산’ 올해보다 8~9% 늘린다,정부가 내년 예산안도 확장 재정 기조를 유지하기로 했다. 올해 본예산 대비 8~9%...
50668,2020-08-26,사회,내일신문,중 경증 분리 절실 “대기업 금융권 나서면 큰 힘”,코로나19 수도권 환자 폭증에 따라 병상 부족 문제가 시급한 과제로 떠올랐다. 대기...
50669,2020-08-26,지역,내일신문,광화문집회 참가자 수사 속도내다,코로나19 새로운 감염원으로 지목된 광화문 집회 참가자에 대한 경찰 수사가 속도를 ...
50670,2020-08-26,사회,내일신문,"[어느 지역병원장의 편지] 포스트 코로나, 그 중심에는 국민이 있어야 한다",지역에서 병원을 운영하는 모 병원장이 내일신문에 편지를 보내왔다. 의사협회의 파업과...


In [164]:
total.to_excel('article.xlsx')